# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [1]:
import datetime
import os 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [2]:
df = pd.read_csv('./data/step3_output.csv')
df.head().T #Transposed to show column names

,0,1,2,3,4
Name,Hilltop Ski Area,Arizona Snowbowl,Sunrise Park Resort,Yosemite Ski & Snowboard Area,Bear Valley
state,Alaska,Arizona,Arizona,California,California
summit_elev,2090,11500,11100,7800,8500
vertical_drop,294,2300,1800,600,1900
trams,0,0,0,0,0
fastEight,0,0,0,0,0
fastSixes,0,1,0,0,1
fastQuads,0,0,1,0,1
quad,0,2,2,0,0
triple,1,2,3,1,2


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort, which you don't need in the dataframe for modeling so it can be droppped here as well. You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [3]:
# Define response variables and features as lists
res_vars = ['AdultWeekend', 'AdultWeekday', 
            'daysOpenLastYear', 'projectedDaysOpen']
cat_vars = ['Name', 'state', 'cluster']
num_vars = list(set(df.columns) - set(res_vars) - set(cat_vars))


# Process numeric variables
from sklearn import preprocessing
df[num_vars] = preprocessing.scale(df[num_vars])

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [4]:
## Process categorical variables
# Remove 'Name' varaiable
# Replace 'state' and 'cluster' with indicator variables
df = pd.concat([df.drop(cat_vars, axis=1), # Drop Categorical vars 
                pd.get_dummies(df.state, prefix='state'), # indicator column for state
                pd.get_dummies(df.cluster, prefix='cluster')], # indicator var for cluster
               axis=1)

## Split into training and testing datasets

**<font color='teal'> Using train_test_split from sklearn.model_selection module, create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [5]:
from sklearn.model_selection import train_test_split

# test_size will be set to 0.25 if neither of test and train size is specified
train, test = train_test_split(df)

In [6]:
# Confirm the split
len_train, len_test = len(train), len(test)
rat_train, rat_test = len(train)/len(df), len(test)/len(df)

print('train: {:>3} entries ({:.2%})\n test: {:>3} entries ({:.2%})'\
      .format(len_train, rat_train, len_test, rat_test ))

train: 139 entries (74.73%)
 test:  47 entries (25.27%)


Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [7]:
# Import modules and functions
from sklearn import linear_model
from sklearn.metrics import (
    explained_variance_score, 
    mean_absolute_error
)

In [8]:
# Define Series for the train and test response values
res_var = 'AdultWeekend'
y_train = train[res_var]
y_test = test[res_var]

In [9]:
# Model 1: linear regression model without tweaking parameters

# Define features to use in the model
feat_1 = [v for v in df.columns 
          if (v not in res_vars) & (not v.startswith('cluster_'))]

# Build a liner regression model
model_1 = linear_model.LinearRegression().fit(train[feat_1], y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [10]:
y_pred_1 = model_1.predict(test[feat_1])
evs_1 = explained_variance_score(y_test, y_pred_1)
mae_1 = mean_absolute_error(y_test, y_pred_1)

print('Explained Variance Score: {:.3f}\n' 
      'Mean Absolute Error     : {:.3f}'.format(evs_1, mae_1))

Explained Variance Score: 0.190
Mean Absolute Error     : 10.939


## Review Model Outcomes — Iterate over additional models as needed.

**<font color='teal'> Print the intercept value from the linear model. </font>**

In [11]:
print('Model1 intercept: {: .3f}'.format(model_1.intercept_))

Model1 intercept:  33658714850963.973


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [12]:
def print_feats_and_coefs(features, model):
    '''Prints feature names and their coefficients in descending order'''

    coefs = dict(zip(features, model.coef_))

    # To iterate and print feature names and their coefficients,
    # create a list of tuples contains absolute value of coef,
    # feature name, and coef.
    abs_coefs= [(abs(coef), feat, coef) for feat, coef in coefs.items()]

    print('Features and Coefficients (Desc order)')
    print('-'*44)
    
    i = 1
    for v in sorted(abs_coefs, reverse=True):
        if i <= 20:
            print('{:>2} {:<23}:{:> 1.3f}'.format(i, v[1], v[2]))
            i += 1
        else:
            print('...')
            break

In [13]:
print_feats_and_coefs(feat_1, model_1)

Features and Coefficients (Desc order)
--------------------------------------------
 1 fastEight              : 1137261104105837.500
 2 total_chairs           :-751229288721908.375
 3 surface                : 399435748740031.250
 4 double                 : 389952207010978.875
 5 triple                 : 325968326494350.812
 6 fastQuads              : 220902949049891.688
 7 quad                   : 207890633285360.656
 8 fastSixes              : 78969349544323.953
 9 trams                  : 41242972974163.195
10 state_Alaska           :-33658714850930.547
11 state_South Dakota     :-33658714850924.219
12 state_Montana          :-33658714850923.953
13 state_Oregon           :-33658714850923.316
14 state_Idaho            :-33658714850920.156
15 state_Ohio             :-33658714850918.523
16 state_Wyoming          :-33658714850917.609
17 state_New Mexico       :-33658714850916.750
18 state_Washington       :-33658714850915.359
19 state_California       :-33658714850912.938
20 state_Iowa  

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [14]:
# Build Model 2
feat_2 = [f for f in feat_1 if not f.startswith('state_')]
model_2 = linear_model.LinearRegression().fit(train[feat_2], y_train)

In [15]:
# Score the performance of Model 2
y_pred_2 = model_2.predict(test[feat_2])
evs_2 = explained_variance_score(y_test, y_pred_2)
mae_2 = mean_absolute_error(y_test, y_pred_2)

print('Explained Variance Score: {: .3f}\n' 
      'Mean Absolute Error     : {: .3f}\n'
      'Intercept               : {: .3f}\n'\
      .format(evs_2, mae_2, model_2.intercept_))
print_feats_and_coefs(feat_2, model_2)

Explained Variance Score:  0.576
Mean Absolute Error     :  8.228
Intercept               :  58.406

Features and Coefficients (Desc order)
--------------------------------------------
 1 vertical_drop          : 5.113
 2 SkiableTerrain_ac      :-3.807
 3 Runs                   : 3.175
 4 fastSixes              : 2.509
 5 fastQuads              : 2.386
 6 triple                 : 2.097
 7 surface                :-1.686
 8 total_chairs           : 1.664
 9 LongestRun_mi          : 1.384
10 Snow Making_ac         : 1.283
11 summit_elev            :-1.280
12 quad                   : 1.263
13 averageSnowfall        : 1.198
14 NightSkiing_ac         : 0.960
15 TerrainParks           :-0.698
16 double                 : 0.685
17 trams                  :-0.371
18 yearsOpen              :-0.210
19 fastEight              :-0.000


**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [16]:
# Build Model 3
feat_3 = [v for v in feat_2 
          if (v != 'summit_elev') & (v != 'vertical_drop')]
model_3 = linear_model.LinearRegression().fit(train[feat_3], y_train)

In [17]:
# Score the performance of Model 3
y_pred_3 = model_3.predict(test[feat_3])
evs_3 = explained_variance_score(y_test, y_pred_3)
mae_3 = mean_absolute_error(y_test, y_pred_3)
print('Explained Variance Score: {: .3f}\n' 
      'Mean Absolute Error     : {: .3f}\n'
      'Intercept               : {: .3f}\n'\
      .format(evs_3, mae_3, model_3.intercept_))
print_feats_and_coefs(feat_3, model_3)

Explained Variance Score:  0.509
Mean Absolute Error     :  8.543
Intercept               :  58.483

Features and Coefficients (Desc order)
--------------------------------------------
 1 Runs                   : 4.076
 2 SkiableTerrain_ac      :-2.854
 3 fastSixes              : 2.801
 4 LongestRun_mi          : 2.774
 5 fastQuads              : 2.677
 6 surface                :-2.013
 7 Snow Making_ac         : 1.947
 8 triple                 : 1.864
 9 total_chairs           : 1.606
10 averageSnowfall        : 1.548
11 quad                   : 1.401
12 double                 : 0.781
13 NightSkiing_ac         : 0.687
14 TerrainParks           :-0.428
15 yearsOpen              :-0.268
16 trams                  :-0.147
17 fastEight              :-0.000


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

In [18]:
print('Model   | {:^18} | {:^19} | Features dropped'.format('Explained Variance', 'Mean Absolute Error'))
print('Model 1 | {:18.3f} | {:19.3f} | -'.format(evs_1, mae_1))
print('Model 2 | {:18.3f} | {:19.3f} |\'state\''.format(evs_2, mae_2))
print('Model 3 | {:18.3f} | {:19.3f} |\'state\',\'summit_elev\', \'base_elev\''.format(evs_3, mae_3))

Model   | Explained Variance | Mean Absolute Error | Features dropped
Model 1 | -94810581495479042834432.000 |    716156846239.053 | -
Model 2 |              0.576 |               8.228 |'state'
Model 3 |              0.509 |               8.543 |'state','summit_elev', 'base_elev'


Model Selection: Model 2

In [19]:
# Optional works

## 1. Check p-values
import statsmodels.api as sm

# Model 1
x_train_1 = train[feat_1].copy()
x_train_1['const'] = 1
result_1 = sm.OLS(y_train, x_train_1).fit().summary()

# Model 2
x_train_2 = train[feat_2].copy()
x_train_2['const'] = 1
result_2 = sm.OLS(y_train, x_train_1).fit().summary()

# Model 3
x_train_3 = train[feat_3].copy()
x_train_3['const'] = 1
result_3 = sm.OLS(y_train, x_train_3).fit().summary()

# print(result_2)

In [20]:
## 2. Utilize categorical variables
# Model A: Utilize cluster instead of state
feat_a = list(set(num_vars) | set(cluster_vars))
feat_a = [f for f in feat_a if not f.startswith('state_')]

x_train_a = train[feat_a]
model_a = sm.OLS(y_train, x_train_a).fit()
model_a.summary()

model_a2 = linear_model.LinearRegression().fit(x_train_a, y_train)
evs, mae = perf_score(model_a2, test[feat_a])
print('Explained Variance Score: {0: .3f}\n'\
      'Mean Absolute Error     : {1: .3f}'.format(evs, mae))

NameError: name 'cluster_vars' is not defined